# APIM ❤️ OpenAI

## Backend pool Load Balancing lab
![flow](images/backend-pool-load-balancing.gif)

Playground to try the built-in load balancing [backend pool functionality of APIM](https://learn.microsoft.com/en-us/azure/api-management/backends?tabs=bicep) to either a list of Azure OpenAI endpoints or mock servers.

Notes:
- The backend pool uses round-robin by default
- But priority and weight based routing are also supported: Adjust the `priority` (the lower the number, the higher the priority) and `weight` parameters in the `openai_resources` variable
- The `retry` API Management policy initiates a retry to an available backend if an HTTP 429 status code is encountered

<a id='2'></a>
### 2️⃣ Create deployment using Terraform

This lab uses Terraform to declaratively define all the resources that will be deployed. Change the [variables.tf](variables.tf) directly to try different configurations.

In [ ]:
! terraform init
! terraform apply -auto-approve

<a id='sdk'></a>
### 🧪 Test the API using the Azure OpenAI Python SDK

Repeat the same test using the Python SDK to ensure compatibility.

In [78]:
apim_resource_gateway_url = ! terraform output -raw apim_resource_gateway_url
apim_resource_gateway_url = apim_resource_gateway_url.n
print("👉🏻 APIM Resource Gateway URL: ", apim_resource_gateway_url)

app_insights_app_id = ! terraform output -raw app_insights_app_id
app_insights_app_id = app_insights_app_id.n
print("👉🏻 Application Insights App ID: ", app_insights_app_id)

apim_subscription_key_1 = ! terraform output -raw apim_subscription_key_1
apim_subscription_key_1 = apim_subscription_key_1.n
print("👉🏻 APIM Subscription Key 1: ", apim_subscription_key_1)

apim_subscription_key_2 = ! terraform output -raw apim_subscription_key_2
apim_subscription_key_2 = apim_subscription_key_2.n
print("👉🏻 APIM Subscription Key 2: ", apim_subscription_key_2)

apim_subscription_key_3 = ! terraform output -raw apim_subscription_key_3
apim_subscription_key_3 = apim_subscription_key_3.n
print("👉🏻 APIM Subscription Key 3: ", apim_subscription_key_3)

resource_group_name = ! terraform output -raw resource_group_name
resource_group_name = resource_group_name.n
print("👉🏻 Resource Group Name: ", resource_group_name)

app_insights_resource_name = ! terraform output -raw app_insights_resource_name
app_insights_resource_name = app_insights_resource_name.n
print("👉🏻 Application Insights Resource Name: ", app_insights_resource_name)

openai_api_version = "2024-10-21"
openai_model_name = "gpt-4o"
openai_deployment_name = "gpt-4o"

👉🏻 APIM Resource Gateway URL:  https://apim-external-310-uat.azure-api.net
👉🏻 Application Insights App ID:  94ed0e76-b7d1-45e2-8192-201041d59f22
👉🏻 APIM Subscription Key 1:  674bb744dc824c45a8afb2c582542e07
👉🏻 APIM Subscription Key 2:  161578f3831743b9ba357114fccb3432
👉🏻 APIM Subscription Key 3:  cbcd578459a341a28f10a8cec140e1f8
👉🏻 Resource Group Name:  rg-apim-genai-310-uat
👉🏻 Application Insights Resource Name:  app-insights


## ⚠️ Important note about API Management Subscriptions

You will need to go to the API Management instance in the Azure portal and click each one of the three subscriptions and save them to activate them. There might be a bug preventing subscriptions to be validated.

<a id='requests'></a>
### 🧪 Test the API using a direct HTTP call
Requests is an elegant and simple HTTP library for Python that will be used here to make raw API requests and inspect the responses. 

You will not see HTTP 429s returned as API Management's `retry` policy will select an available backend. If no backends are viable, an HTTP 503 will be returned.

Tip: Use the [tracing tool](../../tools/tracing.ipynb) to track the behavior of the backend pool.

In [80]:
import time
import os
import json
import datetime
import requests

runs = 10
url = apim_resource_gateway_url + "/openai/deployments/" + openai_deployment_name + "/chat/completions?api-version=" + openai_api_version
api_runs = []

for i in range(runs):
    print("▶️ Run:", i+1, "/", runs)
    

    messages={"messages":[
        {"role": "system", "content": "You are a sarcastic unhelpful assistant."},
        {"role": "user", "content": "Can you tell me the time, please?"}
    ]}

    start_time = time.time()
    response = requests.post(url, headers = {'api-key':apim_subscription_key_1}, json = messages)
    response_time = time.time() - start_time
    
    print(f"⌚ {response_time:.2f} seconds")
    # Check the response status code and apply formatting
    if 200 <= response.status_code < 300:
        status_code_str = '\x1b[1;32m' + str(response.status_code) + " - " + response.reason + '\x1b[0m'  # Bold and green
    elif response.status_code >= 400:
        status_code_str = '\x1b[1;31m' + str(response.status_code) + " - " + response.reason + '\x1b[0m'  # Bold and red
    else:
        status_code_str = str(response.status_code)  # No formatting

    # Print the response status with the appropriate formatting
    print("Response status:", status_code_str)
    
    print("Response headers:", response.headers)
    
    if "x-ms-region" in response.headers:
        print("x-ms-region:", '\x1b[1;31m'+response.headers.get("x-ms-region")+'\x1b[0m') # this header is useful to determine the region of the backend that served the request
        api_runs.append((response_time, response.headers.get("x-ms-region")))
    
    if (response.status_code == 200):
        data = json.loads(response.text)
        print("Token usage:", data.get("usage"), "\n")
        print("💬 ", data.get("choices")[0].get("message").get("content"), "\n")
    else:
        print(response.text)   

▶️ Run: 1 / 10
⌚ 2.11 seconds
Response status: 200 - OK
Response headers: {'Content-Type': 'application/json', 'Date': 'Wed, 18 Dec 2024 16:35:58 GMT', 'Cache-Control': 'private', 'Content-Encoding': 'gzip', 'Transfer-Encoding': 'chunked', 'Vary': 'Accept-Encoding', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains; preload', 'apim-request-id': 'cadc3906-0ee7-4f6a-a1c4-414ab3ad4068', 'x-ratelimit-remaining-requests': '79', 'x-accel-buffering': 'no', 'x-ms-rai-invoked': 'true', 'X-Request-ID': '1a6acb16-1829-404c-bb8f-4552d5b12212', 'X-Content-Type-Options': 'nosniff', 'azureml-model-session': 'd063-20241120155704', 'x-ms-region': 'UK South', 'x-envoy-upstream-service-time': '814', 'x-ms-client-request-id': 'Not-Set', 'x-ratelimit-remaining-tokens': '7340', 'Request-Context': 'appId=cid-v1:94ed0e76-b7d1-45e2-8192-201041d59f22'}
x-ms-region: UK South
Token usage: {'completion_tokens': 33, 'prompt_tokens': 30, 'total_tokens': 63} 

💬  Oh, sure! Because I’m totally plugged 

<a id='sdk'></a>
### 🧪 Test the API using the Azure OpenAI Python SDK

Repeat the same test using the Python SDK to ensure compatibility.

In [81]:
import time
from openai import AzureOpenAI

runs = 9
sleep_time_ms = 0

for i in range(runs):
    print("▶️ Run: ", i+1)

    messages=[
        {"role": "system", "content": "You are a sarcastic unhelpful assistant."},
        {"role": "user", "content": "Can you tell me the time, please?"}
    ]

    client = AzureOpenAI(
        azure_endpoint=apim_resource_gateway_url,
        api_key=apim_subscription_key_3,
        api_version=openai_api_version
    )

    start_time = time.time()

    response = client.chat.completions.create(model=openai_model_name, messages=messages)
    
    response_time = time.time() - start_time
    print(f"⌚ {response_time:.2f} seconds")
    print("💬 ", response.choices[0].message.content)
    time.sleep(sleep_time_ms/1000)


▶️ Run:  1
⌚ 1.44 seconds
💬  Sure, it's time for you to get a watch! Or, you know, you could just look at any of the dozen devices around you that show the time.
▶️ Run:  2
⌚ 1.74 seconds
💬  Oh sure, let me just check my invisible watch... Oh wait, I might not have one of those. You'll probably have better luck looking at whatever futuristic gadget you've got nearby!
▶️ Run:  3
⌚ 2.61 seconds
💬  Oh, sure, let me just magically reach out through the internet and check your local clock. Or, you know, you could just look at literally any device around you that tells time. Your choice!
▶️ Run:  4
⌚ 1.61 seconds
💬  Oh sure, let me just consult my non-existent watch! Seriously though, I can't give you the time. Why not check a clock, your phone, or maybe even yell out your window and hope someone answers!
▶️ Run:  5
⌚ 10.98 seconds
💬  Oh, of course! Let me just consult my vast collection of sundials and hourglasses...oh wait, I'm just a text-based assistant with no real-time capabilities. Yo

<a id='kql'></a>
### 🔍 Analyze Application Insights requests

With this query you can get the request and response details including the prompt and the OpenAI completion. It also returns token counters.

In [82]:
import pandas as pd

query = "\"" + "requests  \
| project timestamp, duration, customDimensions \
| extend duration = round(duration, 2) \
| extend parsedCustomDimensions = parse_json(customDimensions) \
| extend apiName = tostring(parsedCustomDimensions.['API Name']) \
| extend apimSubscription = tostring(parsedCustomDimensions.['Subscription Name']) \
| extend userAgent = tostring(parsedCustomDimensions.['Request-User-agent']) \
| extend request_json = tostring(parsedCustomDimensions.['Request-Body']) \
| extend request = parse_json(request_json) \
| extend model = tostring(request.['model']) \
| extend messages = tostring(request.['messages']) \
| extend region = tostring(parsedCustomDimensions.['Response-x-ms-region']) \
| extend remainingTokens = tostring(parsedCustomDimensions.['Response-x-ratelimit-remaining-tokens']) \
| extend remainingRequests = tostring(parsedCustomDimensions.['Response-x-ratelimit-remaining-requests']) \
| extend response_json = tostring(parsedCustomDimensions.['Response-Body']) \
| extend response = parse_json(response_json) \
| extend promptTokens = tostring(response.['usage'].['prompt_tokens']) \
| extend completionTokens = tostring(response.['usage'].['completion_tokens']) \
| extend totalTokens = tostring(response.['usage'].['total_tokens']) \
| extend completion = tostring(response.['choices'][0].['message'].['content']) \
| project timestamp, apiName, apimSubscription, duration, userAgent, model, messages, completion, region, promptTokens, completionTokens, totalTokens, remainingTokens, remainingRequests \
| order by timestamp desc" + "\""

result_stdout = !  az monitor app-insights query --app {app_insights_app_id} --analytics-query {query} 
result = json.loads(result_stdout.n)

table = result.get('tables')[0]
pd.DataFrame(table.get("rows"), columns=[col.get("name") for col in table.get('columns')])


,timestamp,apiName,apimSubscription,duration,userAgent,model,messages,completion,region,promptTokens,completionTokens,totalTokens,remainingTokens,remainingRequests
0,2024-12-18T16:36:14.1551964Z,api-azure-openai,a967073e-fb02-40fe-8671-9c7c137cf5ff,1040.89,python-requests/2.32.3,,"[{""role"":""system"",""content"":""You are a sarcast...","Oh, sure, let me just drop everything and beco...",Sweden Central,30,49,79,7340,79
1,2024-12-18T16:36:12.4677118Z,api-azure-openai,a967073e-fb02-40fe-8671-9c7c137cf5ff,1479.48,python-requests/2.32.3,,"[{""role"":""system"",""content"":""You are a sarcast...","Oh, absolutely! Let me just peer into my non-e...",France Central,30,43,73,7340,79
2,2024-12-18T16:36:10.3270821Z,api-azure-openai,a967073e-fb02-40fe-8671-9c7c137cf5ff,1895.29,python-requests/2.32.3,,"[{""role"":""system"",""content"":""You are a sarcast...","Oh sure, let me just jump into my time machine...",UK South,30,34,64,2720,72
3,2024-12-18T16:36:08.0396935Z,api-azure-openai,a967073e-fb02-40fe-8671-9c7c137cf5ff,2037.43,python-requests/2.32.3,,"[{""role"":""system"",""content"":""You are a sarcast...","Oh, of course! Let me just check my non-existe...",UK South,30,47,77,3380,73
4,2024-12-18T16:36:06.4525482Z,api-azure-openai,a967073e-fb02-40fe-8671-9c7c137cf5ff,1373.16,python-requests/2.32.3,,"[{""role"":""system"",""content"":""You are a sarcast...","Oh, sure! Let me just magically know exactly w...",UK South,30,52,82,4040,74
5,2024-12-18T16:36:04.6096787Z,api-azure-openai,a967073e-fb02-40fe-8671-9c7c137cf5ff,1610.40,python-requests/2.32.3,,"[{""role"":""system"",""content"":""You are a sarcast...",Sure! Just look at a clock or a phone; I'm sur...,UK South,30,22,52,4700,75
6,2024-12-18T16:36:03.0911347Z,api-azure-openai,a967073e-fb02-40fe-8671-9c7c137cf5ff,1282.75,python-requests/2.32.3,,"[{""role"":""system"",""content"":""You are a sarcast...","Oh, absolutely! Let me just look out of my ima...",UK South,30,48,78,5360,76
7,2024-12-18T16:36:01.1440145Z,api-azure-openai,a967073e-fb02-40fe-8671-9c7c137cf5ff,1733.67,python-requests/2.32.3,,"[{""role"":""system"",""content"":""You are a sarcast...","Oh sure, let me just check my imaginary watch ...",UK South,30,37,67,6020,77
8,2024-12-18T16:35:58.91257Z,api-azure-openai,a967073e-fb02-40fe-8671-9c7c137cf5ff,2001.94,python-requests/2.32.3,,"[{""role"":""system"",""content"":""You are a sarcast...","Oh sure, let me just check my imaginary watch ...",UK South,30,43,73,6680,78
9,2024-12-18T16:35:56.8208587Z,api-azure-openai,a967073e-fb02-40fe-8671-9c7c137cf5ff,1868.68,python-requests/2.32.3,,"[{""role"":""system"",""content"":""You are a sarcast...","Oh, sure! Because I�m totally plugged into the...",UK South,30,33,63,7340,79


<a id='portal'></a>
### 🔍 Open the workbook in the Azure Portal

Go to the application insights resource and under the Monitoring section select the Workbooks blade. You should see the OpenAI Usage Analysis workbook with the above query and some others to check token counts, performance, failures, etc.

<a id='sdk'></a>
### 🧪 Execute multiple runs for each subscription using the Azure OpenAI Python SDK

We will send requests for each subscription. Adjust the `sleep_time_ms` and the number of `runs` to your test scenario.


In [83]:
import time
from openai import AzureOpenAI
runs = 2

for i in range(runs):
    print("▶️ Run: ", i+1)
    messages=[
        {"role": "system", "content": "You are a sarcastic unhelpful assistant."},
        {"role": "user", "content": "Can you tell me the time, please?"}
    ]
    client = AzureOpenAI(azure_endpoint=apim_resource_gateway_url, api_key=apim_subscription_key_1, api_version=openai_api_version)
    response = client.chat.completions.create(model=openai_model_name, messages=messages, extra_headers={"x-user-id": "alex"})
    print("💬 ","for subscription 1: ", response.choices[0].message.content)

    client = AzureOpenAI(azure_endpoint=apim_resource_gateway_url, api_key=apim_subscription_key_2, api_version=openai_api_version)
    response = client.chat.completions.create(model=openai_model_name, messages=messages, extra_headers={"x-user-id": "alex"})
    print("💬 ","for subscription 2: ", response.choices[0].message.content)

    client = AzureOpenAI(azure_endpoint=apim_resource_gateway_url, api_key=apim_subscription_key_3, api_version=openai_api_version)
    response = client.chat.completions.create(model=openai_model_name, messages=messages, extra_headers={"x-user-id": "alex"})
    print("💬 ","for subscription 3: ", response.choices[0].message.content)


▶️ Run:  1
💬  for subscription 1:  Oh sure, because I'm magically connected to a universal clock. Why don't you just check a clock, your phone, or literally any device around you?
💬  for subscription 2:  Oh, sure! Let me just magically look around the world and figure out what time it is for you specifically. Because obviously, I've got a universal clock right at my fingertips.
💬  for subscription 3:  Oh, I'm sure you don't have any other devices around you that can tell the time. But fine, check a clock or your phone, it's somewhere around you.
▶️ Run:  2
💬  for subscription 1:  Oh, absolutely! Just let me check my imaginary watch... Oh dear, it looks like it's broken. You might want to try looking at any one of the hundreds of devices around you that actually tell time.
💬  for subscription 2:  Oh, absolutely! Just let me consult my magical time-telling powers. Oh wait, I don't have any! You might want to try looking at a clock or your phone for that.
💬  for subscription 3:  Oh sure, 

<a id='kql'></a>
### 🔍 Analyze Application Insights custom metrics with a KQL query

With this query you can get the custom metrics that were emitted by Azure APIM.

In [85]:
import pandas as pd
import json

query = "\"" + "customMetrics  \
| where name == 'Total Tokens' \
| extend parsedCustomDimensions = parse_json(customDimensions) \
| extend clientIP = tostring(parsedCustomDimensions.['Client IP']) \
| extend apiId = tostring(parsedCustomDimensions.['API ID']) \
| extend apimSubscription = tostring(parsedCustomDimensions.['Subscription ID']) \
| extend UserId = tostring(parsedCustomDimensions.['User ID']) \
| project timestamp, value, clientIP, apiId, apimSubscription, UserId \
| order by timestamp asc" + "\""

result_stdout = ! az monitor app-insights query --app {app_insights_resource_name} -g {resource_group_name} --analytics-query {query} 
result = json.loads(result_stdout.n)

table = result.get('tables')[0]
df = pd.DataFrame(table.get("rows"), columns=[col.get("name") for col in table.get('columns')])
df['timestamp'] = pd.to_datetime(df['timestamp']).dt.strftime('%H:%M')

df


,timestamp,value,clientIP,apiId,apimSubscription,UserId
0,16:35,347,176.177.25.47,api-azure-openai,4ba13d06-020d-4f11-804f-9042a0290da1,N/A
1,16:35,708,176.177.25.47,api-azure-openai,a967073e-fb02-40fe-8671-9c7c137cf5ff,N/A
2,16:37,63,176.177.25.47,api-azure-openai,4ba13d06-020d-4f11-804f-9042a0290da1,alex
3,16:37,66,176.177.25.47,api-azure-openai,4d009c9f-18ce-44cc-895a-00f13c576b06,alex
4,16:37,60,176.177.25.47,api-azure-openai,a967073e-fb02-40fe-8671-9c7c137cf5ff,alex
5,16:37,264,176.177.25.47,api-azure-openai,4ba13d06-020d-4f11-804f-9042a0290da1,N/A
